In [201]:
import pandas as pd
import random
import scipy as sp
import numpy as np
import cvxpy as cvx
import mosek
import math

#### Load Data

In [202]:
dfTripData = pd.read_csv("location_data.csv")
dfLoadData=pd.read_csv("use_data_2wk.csv") #365 days, 1 min increment, 142 houses 
dfGenData=pd.read_csv("annual_capfac_15MinInterpolation.csv") #representative solar day, 15 min interval

#### Script Parameters

In [213]:
intNumHouseNodes = 5
intNumCommNodes = 2
intAvgNumCarsNode = 5 # goal is to have about 5 * 5 total cars
intSdNumCarsNode = 1
intProfiles = dfTripData.shape[0]

# could make battery parameters a randomized instantiation
fltCarEff = 3.95 # mi/kWh
fltBatteryCap = 40. # kWh
fltChargeRate = 1.9 # kW
fltDt = 0.25 # delta T
fltHomeRate = 1.4 # kW
fltWorkRate = 7 # kW

#time and conversion intervals
int_minutes=int(15)
int_run_days=1 #day
int_run_hours=24 #hours
int_run_time_interval=60/int_minutes #
time_kwh=15/60  #convert kw data to kwh

#### Functions

In [226]:
#process use data to be on 15 minute interals and for specified run time, and associate with nodes

def convertLoadData(dfLoadData,run_days,int_minutes, house_nodes,commercial_nodes):
    minutes=60*24
    run_length=minutes*run_days
    dfLoadData = dfLoadData.dropna(axis=1,thresh=100) #pull only nonzeros columns, but all of them have like one so just set a threshhold
    npLoadData=dfLoadData.to_numpy() #I think iteration is faster in an np array?
    run_loads=npLoadData[0:run_length,2:]    # full run lenth extracted from dataset, 130+ loads, skip timestep/day
    nLoads=len(run_loads[0,:]) 
    resize=int(run_length/int_minutes)
    house_loads=np.zeros((resize,house_nodes))
    commercial_loads=np.zeros((resize,commercial_nodes))
    dfNodeLoads=pd.DataFrame()
    #process to 15 minutes
    i=0 #columns in the node dataframe
    for n in range(house_nodes):  
        random_select=random.randrange(0,nLoads,1)
        house_load_1min=np.sum(run_loads[:,random_select:(random_select+5)],axis=1) #5 house sum per node
        o=int(0)
        j=0
        while o < (run_length-int_minutes): 
            start=o
            end=o+int_minutes
            house_load_1min=np.array(house_load_1min,dtype="float32")
            new_interval=np.nanmean(house_load_1min[start:end])
            house_loads[j,n]=new_interval
            o+=int_minutes
            j+=1
        dfNodeLoads.insert(i,"ResNode"+str(n),house_loads[:,n],True)
        i+=1
    for k in range(commercial_nodes):  #home x10 for commercial, should replace with real commercial values! 
        random_select=random.randrange(0,nLoads,1)
        comm_load_1min=50*run_loads[:,random_select]
        o=int(0)
        j=0
        while o < (run_length-int_minutes): 
            start=o
            end=o+int_minutes
            comm_load_1min=np.array(comm_load_1min,dtype="float32")
            new_interval=np.nanmean(comm_load_1min[start:end])
            commercial_loads[j,k]=new_interval
            o+=int_minutes
            j+=1
        dfNodeLoads.insert(i,"CommNode"+str(k),commercial_loads[:,k],True) 
        i+=1    
    return dfNodeLoads
        
    

In [215]:
def SolarGenData(dfGenData,run_days,int_minutes, house_nodes,commercial_nodes):
    #gendata is capfac at 15 minute intervals for 24 hours 
    intervals=int(60/int_minutes*24) #intervals
    run_length=int(intervals*run_days)
    npGenData=dfGenData.to_numpy() #I think iteration is faster in an np array?
    npGenData=npGenData[:,2]
    house_gen=np.zeros((run_length,house_nodes))
    #every household must average 1 kw generation 
    house_sol_scale=1 #kw
    comm_gen=np.zeros((run_length,commercial_nodes))
    comm_sol_scale=10  #kw,assume 10x gen for solar gen at commercial
    dfNodeGens=pd.DataFrame()
    #columns in the dataframe
    i=0
    for n in range(house_nodes):  
        for d in range(run_days):
            start=0
            end=start+intervals
            house_gen[start:end,n]=np.multiply(npGenData[:],(house_sol_scale*5)) #5 house sum per node
        dfNodeGens.insert(i,"ResNode"+str(n),house_gen[:,n],True)
        i+=1
    for k in range(commercial_nodes):  #home x10 for commercial, should replace with real commercial values! 
        for d in range(run_days):
            start=0
            end=start+intervals
            comm_gen[start:end,k]=npGenData[:]*comm_sol_scale #5 house sum per node
        dfNodeGens.insert(i,"CommNode"+str(k),comm_gen[:,k],True)
        i+=1
    return dfNodeGens

In [216]:
# create residential nodes

def createCarsNodes(intNumHouseNodes, intNumCommNodes, intAvgNumCarsNode, intSdNumCarsNode, \
                    intProfiles, dfTripData):
    #initialize data frame to hold master
    dfNodesTrips = pd.DataFrame(columns=['TripId','ResNode','CommNode'])  

    for intNode in range(intNumHouseNodes):

        # find how many vehicles will be at that residential node
        intNodeNumCars = int(random.gauss(intAvgNumCarsNode,intSdNumCarsNode))
        # assign cars to that node
        lsCarIds = [] # container to hold trip id for that node
        lsCommNode = [] # container to hold where the commerical node 
        lsResNode = [] # container to hold the res node
        for intCar in range(intNodeNumCars):

            # find a car from the data frame
            ind = random.randint(0,intProfiles)
            lsCarIds.append(dfTripData['ID'].iloc[ind]) # add to car id container

            # save res node
            lsResNode.append(intNode)

            # find which commercial node that car goes to
            lsCommNode.append(random.randint(1,intNumCommNodes))

        # create data frame and add to master
        dfNode = pd.DataFrame(list(zip(lsCarIds,lsResNode,lsCommNode)), \
                              columns=['TripId','ResNode','CommNode'])

        dfNodesTrips = dfNodesTrips.append(dfNode)

    dfNodesTrips.reset_index(inplace=True, drop=True)
    
    return dfNodesTrips

    

In [217]:
def findVehicleConsumption(dfNodesTrips, fltCarEff, fltDt):

    '''
    Function to find the power consumption of the vehicle at each time step
    from driving
    '''

    intNumProfs = dfNodesTrips.shape[0]
    arrConsumption = np.zeros((intNumProfs, 96))

    for ind, strId in enumerate(dfNodesTrips['TripId']):

        # find miles travelled
        fltMiles = dfTripData.loc[dfTripData['ID'] == strId,'DIST_M'].values[0]

        # get a list of locations
        lsLocations = dfTripData.loc[dfTripData['ID'] == strId].values.flatten()[3:-1].tolist()

        # so that we can find how many intervals we are away
        intAway = sum(1 if strLoc == "A" else 0 for strLoc in lsLocations)

        # miles per interval of travel and kwh per interval
        fltMiPerInt = fltMiles / intAway
        fltEnergyInt = fltMiPerInt / fltCarEff / fltDt # miles / (miles/kWh) / 0.25 hrs

        lsCarEnergy = [fltEnergyInt*-1 if strLoc == 'A' else 0 for strLoc in lsLocations ] 

        arrConsumption[ind,:] = lsCarEnergy
    
    return arrConsumption
    

In [218]:
def getSOCconstraints(dfNodesTrips, fltCarEff, fltChargeRate, fltDt):

    intNumProfs = dfNodesTrips.shape[0]
    arrSOCrequirements = np.zeros((intNumProfs, 96))

    for ind, strId in enumerate(dfNodesTrips['TripId']):

        # find miles travelled
        fltMiles = dfTripData.loc[dfTripData['ID'] == strId,'DIST_M'].values[0]

        # get a list of locations
        lsLocations = dfTripData.loc[dfTripData['ID'] == strId].values.flatten()[3:-1].tolist()

        # so that we can find how many intervals we are away
        intAway = sum(1 if strLoc == "A" else 0 for strLoc in lsLocations)

        # and then find the average miles travelled each interval
        fltAvgMileInterval = fltMiles / intAway

        intStart = 0 
        while intStart < 96 and 'A' in lsLocations[intStart:]:

            # Find the first first time we start moving
            intStart = intStart + lsLocations[intStart:].index('A') 

            # from there find when we arrive at a destination
            if 'W' in lsLocations[intStart:] and 'H' in lsLocations[intStart:]:
                intEnd = min(lsLocations[intStart:].index('H'), lsLocations[intStart:].index('W'))
            elif 'W' in lsLocations[intStart:]:
                intEnd = lsLocations[intStart:].index('W')
            elif 'H' in lsLocations[intStart:]:
                intEnd = lsLocations[intStart:].index('H')
            else: # we are going to assume they are driving the rest of the time??
                break
               

            # intervals driving for trip is then (intStart+intEnd +1) - intStart = intEnd =1
            # then we can find miles travelled for first trip 
            fltMilesTraveled = fltAvgMileInterval * (intEnd)
            fltKwhRequirement = fltMilesTraveled / fltCarEff # miles / (miles/kWh)

            # now find SOC required
            fltInitialSoc = fltKwhRequirement

            # so we need to to have the initial SOC at the interval before this time:
            arrSOCrequirements[ind,intStart-1] = fltInitialSoc
            
            # now let's back fill in SOC from there
            intNext = 2
            fltPreviousSOC = fltInitialSoc - fltChargeRate * fltDt
            while fltPreviousSOC > 0:
                
                # fill in the SOC needed in the previous time series
                arrSOCrequirements[ind, intStart - intNext] = fltPreviousSOC
                
                intNext += 1 # for next iteration go to the next previous time step
                fltPreviousSOC = fltPreviousSOC - fltChargeRate * fltDt
           

            intStart += intEnd # increment the starting point for finding the next trip

    return arrSOCrequirements


In [219]:
def getChargeRateConstraints(dfNodesTrips, dfTripData):
    '''
    Function to find the maximum charging rate at each time interval and whether it is
    available to charge. 
    '''

    intNumProfs = dfNodesTrips.shape[0]
    # this will hold the charging rate in kW
    arrChargeRate = np.zeros((intNumProfs, 96)) 
    # this will hold the binary of whether the vehicle can charge
    arrCanCharge = np.zeros((intNumProfs, 96))
    

    for ind, strId in enumerate(dfNodesTrips['TripId']):

        # get location sequence for that trip
        lsLocations = dfTripData.loc[dfTripData['ID'] == strId].iloc[0,3:-1].values

        # iterate throught the locations to get its charge rate and availability 
        for intInterval, strLoc in enumerate(lsLocations):

            if strLoc == 'H': # if at home max charge 
                arrChargeRate[ind, intInterval] = fltHomeRate
                arrCanCharge[ind,intInterval] = 1
            elif strLoc == 'W': # working charge rate
                arrChargeRate[ind, intInterval] = fltWorkRate
                arrChargeRate[ind,intInterval] = 1
            else: # you are moving and can't charge
                arrChargeRate[ind, intInterval] = 0
                arrChargeRate[ind, intInterval] = 0
    
    return arrChargeRate, arrCanCharge

In [220]:
def getInitialSoc(arrSOCconstraint): 
    '''
    Function to get an random initialization of the vehicle's SOC
    '''
    
    intNumVehicles = np.shape(arrSOCconstraint)[0]
    lsInitialSoc = np.zeros((intNumVehicles)) # holder to have the initial SOC of vehicles

    for intVehicle in range(intNumVehicles): # for each vehicle 

        fltSocStart = random.random() * fltBatteryCap
        
        # make sure that the random SOC is > the minimum SOC required
        while fltSocStart < arrSOCconstraint[intVehicle,0]:
            fltSocStart = random.random() * fltBatteryCap
        
        lsInitialSoc[intVehicle] = fltSocStart
            
    arrSOCconstraint[:,0] = lsInitialSoc
    
    return arrSOCconstraint

#### Execute

In [227]:
# call functions to get Nodes Dataframe
dfNodesTrips = createCarsNodes(intNumHouseNodes, intNumCommNodes, intAvgNumCarsNode, intSdNumCarsNode, \
                    intProfiles, dfTripData)

#create df of loads for commercial & residential nodes
dfNodeloads=convertLoadData(dfLoadData,int_run_days,int_minutes,intNumHouseNodes,intNumCommNodes)

#create generation for 15 min increments for each node
dfNodeSolarGen=SolarGenData(dfGenData,int_run_days,int_minutes,intNumHouseNodes,intNumCommNodes)

# now call function to get the SOC constraints array
arrSOCconstraint = getSOCconstraints(dfNodesTrips, fltCarEff, fltChargeRate, fltDt)

# call function to get an array of maximum charging rates
arrChargeRate, arrCanCharge = getChargeRateConstraints(dfNodesTrips, dfTripData)

# call function to get initial SOCs of vehicles
# replaces first column of soc constraint array
arrSOCconstraint = getInitialSoc(arrSOCconstraint)

# call function to get power consumption of the vehicle
arrConsumption = findVehicleConsumption(dfNodesTrips, fltCarEff, fltDt)

#### Define OptimizationFunctions

In [222]:
def make_constraints(arrSOCconstraint, arrChargeRate, arrCanCharge,\
                     fltDt, fltBatteryCap, arrConsumption):
    
    intVehicles = np.shape(arrSOCconstraint)[0]
    intTimes = np.shape(arrSOCconstraint)[1]
    
    # make variables
    varCharge = cvx.Variable((intVehicles,intTimes)) # charging rate of vehicles
    varDischarge = cvx.Variable((intVehicles,intTimes)) # discharging rate of vehicles
    varSOCs = cvx.Variable((intVehicles,intTimes)) # SOCs of the vehicles
    
    # initialzie constraints
    constraints = []
    
    # define the charge rate rate constraints
    constraints += [varCharge >= 0] # positively defined
    constraints += [varDischarge <= 0] # negatively defined
    constraints += [varSOCs >= 0] # can't have negative SOC
    constraints += [varSOCs <= fltBatteryCap] # can't over charge
    
    # define charging limits for each vehicle at each time
    for v in range(intVehicles):
    
        # loop through each time to model charging and SOC constraints
        for t in range(intTimes):
            
            # can only charge if present and max rate is given by array
            constraints += [varCharge[v, t] <= arrChargeRate[v,t]*arrCanCharge[v, t]]
            constraints += [-1*arrChargeRate[v,t]*arrCanCharge[v, t] <= varDischarge[v,t]]
            
            # vehicle's SOC must be greater than the minimum requirement
            constraints += [varSOCs[v,t] >= arrSOCconstraint[v,t]]
            
        
        for t in range(1,intTimes): # loop through from time 1 to end to set SOC
            
            constraints += [varSOCs[v,t]  == varSOCs[v,t-1] + varCharge[v,t]*fltDt + varDischarge[v,t]*fltDt + arrConsumption[v,t]*fltDt]
        
        
    
    return constraints, varCharge, varDischarge, varSOCs
       
    

In [223]:
def make_objectives(constraints, varCharge, varDischarge, varSocs, arrCanCharge, arrChargeRate,\
                    arrSOCconstraint,fltBatteryCap,fltDt, fltRegDownVal, fltRegUpVal,lsElectric):
    
    # define regulation up and down variables
    varRegUp = cvx.Variable((np.shape(arrCanCharge)))
    varRegDown = cvx.Variable((np.shape(arrCanCharge)))
    varNumberOfCycles=(cvx.sum(varCharge, axis=1)+cvx.sum(cvx.abs(varDischarge),axis=1))*fltDt/fltBatteryCap
    varBatteryLife=1-fltAlphaSei*math.exp(-varNumberOfCycles*fltBetaSei*fltDegradation)-(1-fltAlphaSei)*math.exp(-varNumberOfCycles*fltDegradation)
    varDegradationCost=fltCost*(1-varBatteryLife)/0.2
    
    # always positive
    constraints += [varRegUp >= 0]
    constraints += [varRegDown >= 0]
    
    intVehicles = np.shape(arrSOCconstraint)[0]
    intTimes = np.shape(arrSOCconstraint)[1]
    
    for v in range(intVehicles):
        for t in range(intTimes):
            
            # reg down constraints
            # it is at most the charging rate if it's available
            constraints += [varRegDown[v,t] <= arrCanCharge[v,t]*arrChargeRate[v,t]]
            # or what is left to charge batteries capacity
            constraints += [varRegDown[v,t] <= (fltBatteryCap - varSocs[v,t])/fltDt]
            
            # reg up constraints 
            # it is at most the charge rate if it's avalable 
            constraints += [varRegUp[v,t] <= arrCanCharge[v,t]*arrChargeRate[v,t]]
            # or what it's minimum SOC requriement is
            constraints += [varRegUp[v,t] <= (varSocs[v,t] - arrSOCconstraint[v,t])/fltDt]
            
    
    obj_value = cvx.sum(varRegDown - varCharge)*fltRegDownVal + \
                cvx.sum(varRegUp - varDischarge)*fltRegUpVal + \
                cvx.sum(lsElectric*cvx.sum(varCharge,axis=0)) + \
               cvx.sum(varDegradationCost,axis=0)
    
    return constraints, obj_value

In [224]:
# dummy values -- MUST UPDATE
fltRegDownVal=4 
fltRegUpVal= 5
lsElectric = [0 for i in range(96)]

# battery constants
fltAlphaSei = 5.75 * 10**-2
fltBetaSei = 121
fltKdelta1 = 1.40 * 10**5
fltKdelta2 = -5.01 * 10**-1
fltKdelta3= 1.23*10**5
fltKsigma = 1.04
fltSigmaRef = 0.5
fltKtime = 4.14* 10**-10 # 1/s
fltKtemp = 6.93 * 10**-2
fltTref = 25 
fltCost = 8500 # cost of Nissan Leaf battery replacement

fltTemp=25
fltDepthofDischarge=0.6
fltStateofCharge=0.7
fltTime=3600*runHours # time in seconds


fltStressTemp=math.exp(fltKtemp(fltTemp-fltTRef))*fltTRef/fltTemp
fltStressSOC=math.exp(fltKsigma(fltStateofCharge-fltSigmaRef))
fltStressTime=fltKtime(t)
fltStressDischage=(fltKdelta1*fltDepthofDischarge**fltKdelta2+fltKdelta3)**(-1)

fltDegradation=[fltStressDischarge+fltStressTime]*fltStressSOC*fltStressTemp


In [225]:
constraints, varCharge, varDischarge, varSocs = make_constraints(arrSOCconstraint,\
                                        arrChargeRate, arrCanCharge, fltDt, fltBatteryCap,\
                                        arrConsumption)
constraints, obj_value = make_objectives(constraints, varCharge, varDischarge, varSocs, \
                        arrCanCharge, arrChargeRate,arrSOCconstraint,fltBatteryCap,fltDt, \
                                         fltRegDownVal, fltRegUpVal,lsElectric)
    
#convex usually needs to be minimized
obj = cvx.Minimize(-obj_value)
prob = cvx.Problem(obj, constraints)

prob.solve(solver=cvx.MOSEK)

Error: (1008) License cannot be located. The default search path is ';C:\Users\evbga\mosek\mosek.lic;'.